<a href="https://colab.research.google.com/github/Dilshodyorqinovich/AI-prompting/blob/main/Shortestpath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import tracemalloc
from collections import deque
import heapq


In [ ]:
GRID_SIZE = 10
START = (0, 0)
GOAL = (9, 9)

# Example obstacles
OBSTACLES = {
    (1, 2), (2, 2), (3, 2), (4, 2),
    (4, 3), (4, 4), (5, 4), (6, 4),
    (7, 4), (7, 5), (7, 6)
}

MOVES = [(-1,0), (1,0), (0,-1), (0,1)]


In [ ]:
def is_valid(x, y):
    return (0 <= x < GRID_SIZE and
            0 <= y < GRID_SIZE and
            (x, y) not in OBSTACLES)

def heuristic(a, b):
    # Manhattan distance
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


In [ ]:
def bfs():
    tracemalloc.start()
    start_time = time.time()

    queue = deque([START])
    visited = set([START])
    expanded = 0

    while queue:
        current = queue.popleft()
        expanded += 1

        if current == GOAL:
            break

        for dx, dy in MOVES:
            nx, ny = current[0] + dx, current[1] + dy
            if is_valid(nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny))

    elapsed = time.time() - start_time
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    return expanded, elapsed, memory

bfs()


In [ ]:
def dfs():
    tracemalloc.start()
    start_time = time.time()

    stack = [START]
    visited = set([START])
    expanded = 0

    while stack:
        current = stack.pop()
        expanded += 1

        if current == GOAL:
            break

        for dx, dy in MOVES:
            nx, ny = current[0] + dx, current[1] + dy
            if is_valid(nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                stack.append((nx, ny))

    elapsed = time.time() - start_time
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    return expanded, elapsed, memory


dfs()

In [ ]:
def greedy():
    tracemalloc.start()
    start_time = time.time()

    pq = [(heuristic(START, GOAL), START)]
    visited = set([START])
    expanded = 0

    while pq:
        _, current = heapq.heappop(pq)
        expanded += 1

        if current == GOAL:
            break

        for dx, dy in MOVES:
            nx, ny = current[0] + dx, current[1] + dy
            if is_valid(nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                heapq.heappush(pq, (heuristic((nx, ny), GOAL), (nx, ny)))

    elapsed = time.time() - start_time
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    return expanded, elapsed, memory

greedy()

In [ ]:
def astar():
    tracemalloc.start()
    start_time = time.time()

    pq = [(heuristic(START, GOAL), 0, START)]
    visited = set()
    expanded = 0

    while pq:
        _, cost, current = heapq.heappop(pq)

        if current in visited:
            continue

        visited.add(current)
        expanded += 1

        if current == GOAL:
            break

        for dx, dy in MOVES:
            nx, ny = current[0] + dx, current[1] + dy
            if is_valid(nx, ny):
                new_cost = cost + 1
                priority = new_cost + heuristic((nx, ny), GOAL)
                heapq.heappush(pq, (priority, new_cost, (nx, ny)))

    elapsed = time.time() - start_time
    memory = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    return expanded, elapsed, memory

astar()

In [ ]:
algorithms = {
    "BFS": bfs,
    "DFS": dfs,
    "Greedy": greedy,
    "A*": astar
}

for name, algo in algorithms.items():
    expanded, time_used, memory = algo()
    print(f"{name}:")
    print(f"  Expanded states: {expanded}")
    print(f"  Execution time: {time_used:.6f} seconds")
    print(f"  Memory usage: {memory / 1024:.2f} KB\n")


In [ ]:
import matplotlib.pyplot as plt

def visualize_grid(grid_size, start, goal, obstacles, path=None, title="Grid Path"):
    """
    Visualize the grid environment and path.

    Arguments:
    - grid_size : int, size of the grid (grid_size x grid_size)
    - start     : tuple, starting position (x, y)
    - goal      : tuple, goal position (x, y)
    - obstacles : set of tuples, obstacle positions
    - path      : list of tuples, path from start to goal (optional)
    - title     : str, title of the plot
    """

    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim(-0.5, grid_size-0.5)
    ax.set_ylim(-0.5, grid_size-0.5)
    ax.set_xticks(range(grid_size))
    ax.set_yticks(range(grid_size))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True)

    # Draw obstacles
    for (x, y) in obstacles:
        ax.add_patch(plt.Rectangle((y-0.5, grid_size-1-x-0.5), 1, 1, color='black'))

    # Draw start
    ax.add_patch(plt.Rectangle((start[1]-0.5, grid_size-1-start[0]-0.5), 1, 1, color='green', label='Start'))

    # Draw goal
    ax.add_patch(plt.Rectangle((goal[1]-0.5, grid_size-1-goal[0]-0.5), 1, 1, color='red', label='Goal'))

    # Draw path
    if path:
        for (x, y) in path:
            if (x, y) != start and (x, y) != goal:
                ax.add_patch(plt.Rectangle((y-0.5, grid_size-1-x-0.5), 1, 1, color='yellow'))
        # Draw path line
        path_coords = [(y, grid_size-1-x) for (x, y) in path]
        xs, ys = zip(*path_coords)
        ax.plot(xs, ys, color='blue', linewidth=2, marker='o')

    ax.set_title(title)
    ax.legend(handles=[
        plt.Line2D([0], [0], color='green', lw=4, label='Start'),
        plt.Line2D([0], [0], color='red', lw=4, label='Goal'),
        plt.Line2D([0], [0], color='yellow', lw=4, label='Path'),
        plt.Line2D([0], [0], color='black', lw=4, label='Obstacle')
    ], loc='upper right')

    plt.show()


In [ ]:
# Example after running BFS
visualize_grid(
    grid_size=10,
    start=START,
    goal=GOAL,
    obstacles=OBSTACLES,
    path=path,          # path returned by BFS
    title="BFS Path Visualization"
)


NameError: name 'START' is not defined

In [ ]:
visualize_grid(grid_size=10, start=START, goal=GOAL, obstacles=OBSTACLES, path=dfs_path, title="DFS Path")
visualize_grid(grid_size=10, start=START, goal=GOAL, obstacles=OBSTACLES, path=greedy_path, title="Greedy Path")
visualize_grid(grid_size=10, start=START, goal=GOAL, obstacles=OBSTACLES, path=a_star_path, title="A* Path")



NameError: name 'START' is not defined